In [40]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.manifold import TSNE
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.spatial.distance import cosine
import spacy
import pandas as pd
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import networkx as nx
import spacy
from collections import Counter

In [41]:
model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')

# Dataframe work

In [42]:
df_full = pd.read_csv("/Users/merlesteffen/Documents/Education/WBS_Coding_School/Bootcamp/Sentiment_In_Session/Final_DF/sentiment_analysis_all.csv")

In [148]:
df_full['Party'] = df_full['Party'].replace('Bündnis 90/Die Grünen', 'Die Grünen')

# Spacy model

In [49]:
nlp = spacy.load('de_core_news_lg')

# Let us add the Party

In [56]:
df_full['Party'] = df_full['Party'].replace('Bündnis 90/Die Grünen', 'Die Grünen')

In [57]:
df1 = df_full.copy()

In [59]:
# Chunky chunk chunk
# Group by speaker and concatenate all text spoken by the same speaker
grouped_df = df1.groupby('Name')['Text_Spoken'].apply(' '.join).reset_index()

# Add the 'Party' column to the grouped_df
grouped_df = grouped_df.merge(df1[['Name', 'Party']].drop_duplicates(), on='Name')

# Function to process text in chunks
def process_text_in_chunks(text, chunk_size=1000000):
    # Process the text in chunks of the given size
    doc = nlp(text[:chunk_size])
    processed_text = ' '.join([token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct and token.is_alpha])
    for i in range(chunk_size, len(text), chunk_size):
        doc = nlp(text[i:i+chunk_size])
        processed_text += ' ' + ' '.join([token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct and token.is_alpha])
    return processed_text

# Apply preprocessing to the grouped DataFrame
grouped_df['Processed_Text'] = grouped_df['Text_Spoken'].apply(lambda text: process_text_in_chunks(text))

# Vectorize the text
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(grouped_df['Processed_Text'])

# Calculate cosine similarity
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Convert cosine similarity matrix to DataFrame
cosine_sim_df_text = pd.DataFrame(cosine_sim, index=grouped_df['Name'], columns=grouped_df['Name'])

# Display the similarity matrix
cosine_sim_df_text

Name,Achim Großmann,Achim Kessler,Achim Post(Minden),Adis Ahmetovic,Agnes Alpers,Agnes Krumwiede,Agnieszka Brugger,Albert Deß,Albert Rupprecht,Albert Rupprecht(Weiden),...,Wolfgang Zeitlmann,Wolfgang Zöller,Xaver Jung,Yannick Bury,Yasmin Fahimi,Yvonne Magwas,Yvonne Ploetz,Zanda Martens,Zoe Mayer,Özcan Mutlu
Name,,,,,,,,,,,,,,,,,,,,,
Achim Großmann,1.000000,0.086498,0.041813,0.060430,0.089371,0.070962,0.105233,0.086774,0.099447,0.155185,...,0.053512,0.103455,0.100525,0.083583,0.088597,0.056701,0.083136,0.066773,0.054144,0.112386
Achim Kessler,0.086498,1.000000,0.073352,0.196718,0.282239,0.141790,0.358036,0.187459,0.176630,0.317620,...,0.152529,0.438289,0.212067,0.231755,0.261717,0.198766,0.214815,0.185008,0.149216,0.300905
Achim Post(Minden),0.041813,0.073352,1.000000,0.091242,0.085825,0.065385,0.137815,0.083431,0.076621,0.135766,...,0.047100,0.072222,0.100753,0.095925,0.107815,0.080142,0.069290,0.082834,0.071320,0.157573
Adis Ahmetovic,0.060430,0.196718,0.091242,1.000000,0.159102,0.094647,0.373334,0.137971,0.118529,0.216007,...,0.099147,0.130214,0.151346,0.205479,0.186005,0.155225,0.155635,0.150996,0.132334,0.223497
Agnes Alpers,0.089371,0.282239,0.085825,0.159102,1.000000,0.164674,0.327966,0.185153,0.199188,0.350742,...,0.145303,0.220710,0.279640,0.206790,0.434535,0.178626,0.297144,0.164596,0.130399,0.318780
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yvonne Magwas,0.056701,0.198766,0.080142,0.155225,0.178626,0.106656,0.255522,0.178752,0.122343,0.236849,...,0.103822,0.144357,0.143046,0.219418,0.218132,1.000000,0.172512,0.201052,0.212713,0.233518
Yvonne Ploetz,0.083136,0.214815,0.069290,0.155635,0.297144,0.152312,0.296460,0.210905,0.173091,0.309582,...,0.127320,0.212387,0.238837,0.181834,0.261126,0.172512,1.000000,0.171200,0.151346,0.280729
Zanda Martens,0.066773,0.185008,0.082834,0.150996,0.164596,0.110865,0.248675,0.148749,0.114142,0.221163,...,0.109009,0.170108,0.152041,0.195004,0.226308,0.201052,0.171200,1.000000,0.166753,0.214346


In [60]:
# Save the cosine similarity matrix to a CSV file
cosine_sim_df_text.to_csv('/Users/merlesteffen/Documents/Education/WBS_Coding_School/Bootcamp/Sentiment_In_Session/Final_DF/cosine_similarity_matrix.csv')

In [61]:
party_colors_map = {
    'SPD': 'red',
    'PDS': '#801818',
    'CDU': 'white',
    'FDP': 'yellow',
    'Die Grünen': 'green',
    'CSU': 'white',
    'AfD': '#0489DB',
    'Die Linke': 'pink',
    'fraktionslos': 'white',
    'LKR': '#FE8100',
    'SSW': '#00277E'
}

In [84]:
# Initialize the graph
G = nx.Graph()

# Add nodes and associate each node with a party
for index, row in grouped_df.iterrows():
    G.add_node(row['Name'], party=row['Party'])

# Get a list of colors for each node
node_colors = party_colors_map

# Define a threshold for similarity to draw an edge
threshold = 0.00000000000001  # You may need to adjust this value

# Add edges to the graph
for i, speaker1 in enumerate(cosine_sim_df_text.index):
    for j, speaker2 in enumerate(cosine_sim_df_text.columns):
        if i < j:  # This ensures that each pair is only considered once
            similarity = cosine_sim_df_text.iloc[i, j]
            if similarity > threshold:
                G.add_edge(speaker1, speaker2, weight=similarity)

# Draw the graph
plt.figure(figsize=(20, 10), facecolor='#282828')
pos = nx.kamada_kawai_layout(G)
#pos = nx.spring_layout(G, k=0.1)  # k is the optimal distance between nodes, may need to be adjusted

# Create a subgraph of G that only includes the largest connected component
# If you want to include smaller connected components as well, you can adjust this part
largest_cc = max(nx.connected_components(G), key=len)
subgraph = G.subgraph(largest_cc)

# Draw only the nodes and edges from the largest connected component
nx.draw_networkx_nodes(subgraph, pos, node_color=[party_colors_map[subgraph.nodes[speaker]['party']] for speaker in subgraph.nodes], alpha=0.7, node_size=50)  # Adjust node_size as needed
nx.draw_networkx_edges(subgraph, pos, edge_color='gray', alpha=0.5, width=0.5)


# Create a legend for the color map
party_labels = list(party_colors_map.keys())
patches = [plt.Line2D([0], [0], marker='o', color='w', label=party,
                      markersize=10, markerfacecolor=color) for party, color in party_colors_map.items()]
plt.legend(handles=patches, title="Political Parties", loc='upper left', bbox_to_anchor=(1, 1), fontsize='large')

plt.title('Network Graph of Speaker Similarity',  fontsize=20, loc='center', color='white')
plt.axis('off')
plt.savefig('/Users/merlesteffen/Documents/Education/WBS_Coding_School/Bootcamp/Sentiment_In_Session/Präsentation/Graphs/speaker_similarity_graph_6.png', dpi=300, bbox_inches='tight', facecolor='#282828')
plt.show();

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/merlesteffen/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/7h/lk9jxn256x10l_m01pz7khdw0000gn/T/ipykernel_54266/250786080.py", line 24, in <module>
    pos = nx.kamada_kawai_layout(G)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/merlesteffen/anaconda3/lib/python3.11/site-packages/networkx/drawing/layout.py", line 690, in kamada_kawai_layout
    dist = dict(nx.shortest_path_length(G, weight=weight))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/merlesteffen/anaconda3/lib/python3.11/site-packages/networkx/algorithms/shortest_paths/weighted.py", line 1071, in all_pairs_dijkstra_path_length
    yield (n, length(G, n, cutoff=cutoff, weight=weight))
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/merlesteffen/anaconda3/lib/python3.11/site-packages/networkx/algori

<Figure size 2000x1000 with 0 Axes>

# Networkgraph Based on Sentiment

In [63]:
# Group by 'Name' and calculate the mean of each sentiment
grouped_sentiment_df = df1.groupby('Name')[['positive', 'neutral', 'negative']].mean().reset_index()

# Add the 'Party' column to the grouped_df before creating the sentiment matrix
grouped_sentiment_df = grouped_sentiment_df.merge(df1[['Name', 'Party']].drop_duplicates(), on='Name')

# Create the sentiment matrix from the grouped DataFrame
sentiment_matrix = grouped_sentiment_df[['positive', 'neutral', 'negative']].values

# Calculate cosine similarity
cosine_sim_sentiment = cosine_similarity(sentiment_matrix, sentiment_matrix)

# Convert cosine similarity matrix to DataFrame
cosine_sim_df_sentiment = pd.DataFrame(cosine_sim_sentiment, 
                                       index=grouped_sentiment_df['Name'], 
                                       columns=grouped_sentiment_df['Name'])

# Display the similarity matrix
cosine_sim_df_sentiment

Name,Achim Großmann,Achim Kessler,Achim Post(Minden),Adis Ahmetovic,Agnes Alpers,Agnes Krumwiede,Agnieszka Brugger,Albert Deß,Albert Rupprecht,Albert Rupprecht(Weiden),...,Wolfgang Zeitlmann,Wolfgang Zöller,Xaver Jung,Yannick Bury,Yasmin Fahimi,Yvonne Magwas,Yvonne Ploetz,Zanda Martens,Zoe Mayer,Özcan Mutlu
Name,,,,,,,,,,,,,,,,,,,,,
Achim Großmann,1.000000,0.991177,0.997856,0.999803,0.997025,0.998559,0.985098,0.982285,0.999900,0.998903,...,0.963234,0.988974,0.998090,0.996439,0.999330,0.997793,0.995270,0.994128,0.999871,0.995080
Achim Kessler,0.991177,1.000000,0.997720,0.988547,0.996974,0.996810,0.998575,0.997704,0.989959,0.996029,...,0.990115,0.999825,0.982550,0.998558,0.992001,0.985141,0.999348,0.999658,0.993164,0.998716
Achim Post(Minden),0.997856,0.997720,1.000000,0.996477,0.999103,0.999918,0.994024,0.992178,0.997176,0.999658,...,0.978702,0.996540,0.992539,0.999732,0.997835,0.993605,0.999468,0.999032,0.998775,0.998991
Adis Ahmetovic,0.999803,0.988547,0.996477,1.000000,0.995319,0.997432,0.982130,0.979095,0.999774,0.997781,...,0.958369,0.986146,0.998564,0.994867,0.998667,0.997683,0.993254,0.991888,0.999394,0.992921
Agnes Alpers,0.997025,0.996974,0.999103,0.995319,1.000000,0.998848,0.991605,0.989530,0.996820,0.999430,...,0.976764,0.995347,0.993169,0.998178,0.998486,0.995289,0.998911,0.998618,0.997884,0.999586
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yvonne Magwas,0.997793,0.985141,0.993605,0.997683,0.995289,0.994425,0.975673,0.972108,0.998615,0.996210,...,0.951537,0.981859,0.999461,0.990759,0.998879,1.000000,0.990558,0.989293,0.997103,0.992092
Yvonne Ploetz,0.995270,0.999348,0.999468,0.993254,0.998911,0.998972,0.996463,0.995038,0.994408,0.998594,...,0.984421,0.998571,0.988617,0.999620,0.995868,0.990558,1.000000,0.999935,0.996681,0.999623
Zanda Martens,0.994128,0.999658,0.999032,0.991888,0.998618,0.998390,0.997030,0.995748,0.993210,0.997984,...,0.986128,0.999014,0.987036,0.999319,0.994960,0.989293,0.999935,1.000000,0.995708,0.999609


In [64]:
# Save the cosine similarity matrix to a CSV file
cosine_sim_df_sentiment.to_csv('/Users/merlesteffen/Documents/Education/WBS_Coding_School/Bootcamp/Sentiment_In_Session/Final_DF/cosine_sim_df_sentiment.csv')

In [ ]:
# Initialize the graph
G = nx.Graph()

# Add nodes and associate each node with a party
for index, row in grouped_sentiment_df.iterrows():
    G.add_node(row['Name'], party=row['Party'])

# Get a list of colors for each node
node_colors = party_colors_map

# Define a threshold for similarity to draw an edge
threshold = 0.00000000000001  # You may need to adjust this value

# Add edges to the graph
for i, speaker1 in enumerate(cosine_sim_df_sentiment.index):
    for j, speaker2 in enumerate(cosine_sim_df_sentiment.columns):
        if i < j:  # This ensures that each pair is only considered once
            similarity = cosine_sim_df_sentiment.iloc[i, j]
            if similarity > threshold:
                G.add_edge(speaker1, speaker2, weight=similarity)

# Draw the graph
plt.figure(figsize=(20, 10), facecolor='#282828')
pos = nx.kamada_kawai_layout(G)
#pos = nx.spring_layout(G, k=0.1)  # k is the optimal distance between nodes, may need to be adjusted

# Create a subgraph of G that only includes the largest connected component
# If you want to include smaller connected components as well, you can adjust this part
largest_cc = max(nx.connected_components(G), key=len)
subgraph = G.subgraph(largest_cc)

# Draw only the nodes and edges from the largest connected component
nx.draw_networkx_nodes(subgraph, pos, node_color=[party_colors_map[subgraph.nodes[speaker]['party']] for speaker in subgraph.nodes], alpha=0.7, node_size=50)  # Adjust node_size as needed
nx.draw_networkx_edges(subgraph, pos, edge_color='gray', alpha=0.5, width=0.5)


# Create a legend for the color map
party_labels = list(party_colors_map.keys())
patches = [plt.Line2D([0], [0], marker='o', color='w', label=party,
                      markersize=10, markerfacecolor=color) for party, color in party_colors_map.items()]
plt.legend(handles=patches, title="Political Parties", loc='upper left', bbox_to_anchor=(1, 1), fontsize='large')

plt.title('Network Graph of Speaker Similarity',  fontsize=20, loc='center', color='white')
plt.axis('off')
plt.savefig('/Users/merlesteffen/Documents/Education/WBS_Coding_School/Bootcamp/Sentiment_In_Session/Präsentation/Graphs/8.png', dpi=300, bbox_inches='tight', facecolor='#282828')
plt.show();